In [1]:
import os, cv2
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [3]:
def get_landmarks(file):
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.5) as hands:
            img = cv2.flip(cv2.imread(file), 1)

            img_col = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            res = hands.process(img_col)
                        
            
            return res.multi_hand_landmarks

def print_landmarks(lm):
    for hand_landmarks in lm:
        print(hand_landmarks)

def plot_landmarks(lm):
    for hand_world_landmarks in lm:
                mp_drawing.plot_landmarks(
                hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)
        

In [4]:
# Get training images from train_imgs

import random

if not os.path.isdir(os.path.join(os.getcwd(), 'train_imgs')):
    for category in os.listdir(os.path.join(os.getcwd(), 'train_imgs_all')):
        train_path = os.path.join(os.getcwd(), 'train_imgs_all', category)
        new_path = os.path.join(os.getcwd(), 'train_imgs', category)
        if not os.path.isdir(new_path):
            os.mkdir(new_path)
        
        selected = []
        f_mv = '-1'
        
        for i in range(100):
            while f_mv in selected or f_mv == '-1':
                f_mv = random.choice(os.listdir(train_path))
            
            selected.append(f_mv)

            os.rename(
                os.path.join(train_path,f_mv),
                os.path.join(new_path, f_mv)
            )

In [8]:
# Data augmentation - flip files horizontally
for category in os.listdir(os.path.join(os.getcwd(), 'train_imgs')):
    for f_to_read in os.listdir(os.path.join(os.getcwd(), 'train_imgs', category)):
        img_path = os.path.join(os.getcwd(), 'train_imgs', category, f_to_read)
        img_to_flip = cv2.imread(img_path)
        img_flip = cv2.flip(img_to_flip, 1)
        
        flip_path = os.path.join(os.getcwd(), 'train_imgs', category, f_to_read[:-4]+'flip.jpg')
        cv2.imwrite(flip_path, img_flip)

In [22]:
# Data augmentation - rotate files
for category in os.listdir(os.path.join(os.getcwd(), 'train_imgs')):
    for f_to_read in os.listdir(os.path.join(os.getcwd(), 'train_imgs', category)):
        img_path = os.path.join(os.getcwd(), 'train_imgs', category, f_to_read)

        img = cv2.imread(img_path)

        angle = int(random.uniform(-30, 30))
        h, w = img.shape[:2]
        M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
        img = cv2.warpAffine(img, M, (w, h))
        
        rotate_path = os.path.join(os.getcwd(), 'train_imgs', category, f_to_read[:-4]+'r.jpg')
        cv2.imwrite(rotate_path, img)

In [6]:
# PROCESS DATA (Calculate relative pos from lm 0 by subtracting)

for category in os.listdir(os.path.join(os.getcwd(), 'train_imgs')):
    for f_to_read in os.listdir(os.path.join(os.getcwd(), 'train_imgs', category)):
        if not os.path.isdir(os.path.join('train_data', category)):
            os.mkdir(os.path.join('train_data', category))
        
        with open(os.path.join('train_data', category, f_to_read[:-4]+'.txt'), 'w') as f_to_save:    
            lm = get_landmarks(os.path.join(os.getcwd(), 'train_imgs', category, f_to_read))
            
            str_to_save = ''
            
            if not lm == None:
                hand_lm = lm[0].landmark
                
                # PROCESS DATA (Calculate relative pos from lm 0 by subtracting)
                for i, lm in enumerate(hand_lm):
                    if i==0:
                        refx, refy, refz = lm.x, lm.y, lm.z
                    else:
                        print(i, lm)
                        x = lm.x - refx
                        y = lm.y - refy
                        z = lm.z - refz
                        str_to_save += f'{x}|{y}|{z}\n'
                
            f_to_save.write(str_to_save)


1 x: 0.23084178566932678
y: 0.7200640439987183
z: -0.04003393277525902

2 x: 0.28021857142448425
y: 0.6395058631896973
z: -0.05221140757203102

3 x: 0.2438066005706787
y: 0.5614824891090393
z: -0.06323369592428207

4 x: 0.1972135454416275
y: 0.508599579334259
z: -0.07181093096733093

5 x: 0.2866804599761963
y: 0.568972110748291
z: -0.00956420972943306

6 x: 0.3056752681732178
y: 0.481224924325943
z: -0.03624493256211281

7 x: 0.3125133514404297
y: 0.4211307466030121
z: -0.05438198521733284

8 x: 0.3174704909324646
y: 0.3672295808792114
z: -0.06526074558496475

9 x: 0.23671068251132965
y: 0.5549862384796143
z: -0.008932155556976795

10 x: 0.27339407801628113
y: 0.45734140276908875
z: -0.04077663645148277

11 x: 0.3042975664138794
y: 0.40301215648651123
z: -0.05719441547989845

12 x: 0.32728278636932373
y: 0.3548465669155121
z: -0.062022384256124496

13 x: 0.18215861916542053
y: 0.5611518621444702
z: -0.014860795810818672

14 x: 0.19147615134716034
y: 0.49466264247894287
z: -0.0751983150

KeyboardInterrupt: 

In [18]:
# Remove blank data files
for category in os.listdir(os.path.join(os.getcwd(),'train_data')):
    for file in os.listdir(os.path.join(os.getcwd(),'train_data', category)):
        f_read = open(os.path.join(os.getcwd(),'train_data', category, file))
        if f_read.read() == '':
            d_path = os.path.join(os.getcwd(),'train_data', category, file)
            i_path = os.path.join(os.getcwd(),'train_imgs', category, file[:-4]+'.jpg')
            os.remove(d_path)
            os.remove(i_path)

In [19]:
# Move thirty percent of train to test

import random
for category in os.listdir(os.path.join(os.getcwd(), 'train_data')):
    train_path = os.path.join(os.getcwd(), 'train_data', category)
    test_path = os.path.join(os.getcwd(), 'test_data', category)
    if not os.path.isdir(test_path):
        os.mkdir(test_path)
    
    for i in range(round(0.3*len(os.listdir(os.path.join(os.getcwd(), 'train_data'))))):
        f_mv = random.choice(os.listdir(train_path))

        os.rename(
            os.path.join(train_path,f_mv),
            os.path.join(test_path, f_mv)
        )

In [7]:
# Move all test back to train
for category in os.listdir(os.path.join(os.getcwd(), 'test_data')):
    for file in os.listdir(os.path.join(os.getcwd(), 'test_data', category)):
        train_path = os.path.join(os.getcwd(), 'train_data', category)
        test_path = os.path.join(os.getcwd(), 'test_data', category)
        
        os.rename(
            os.path.join(test_path, file),
            os.path.join(train_path, file)
        )